In [1]:
from pyspark import SparkContext, RDD, SparkConf
from math import exp, log

In [2]:
#Set the config for spark to enhance performance
config = SparkConf()\
            .set("spark.driver.memory", "4g")\
            .set("spark.executor.memory", "4g")

[([0.0,
   -1.3598071336738,
   -0.0727811733098497,
   2.53634673796914,
   1.37815522427443,
   -0.338320769942518,
   0.462387777762292,
   0.239598554061257,
   0.0986979012610507,
   0.363786969611213,
   0.0907941719789316,
   -0.551599533260813,
   -0.617800855762348,
   -0.991389847235408,
   -0.311169353699879,
   1.46817697209427,
   -0.470400525259478,
   0.207971241929242,
   0.0257905801985591,
   0.403992960255733,
   0.251412098239705,
   -0.018306777944153,
   0.277837575558899,
   -0.110473910188767,
   0.0669280749146731,
   0.128539358273528,
   -0.189114843888824,
   0.133558376740387,
   -0.0210530534538215,
   149.62],
  0.0),
 ([0.0,
   1.19185711131486,
   0.26615071205963,
   0.16648011335321,
   0.448154078460911,
   0.0600176492822243,
   -0.0823608088155687,
   -0.0788029833323113,
   0.0851016549148104,
   -0.255425128109186,
   -0.166974414004614,
   1.61272666105479,
   1.06523531137287,
   0.48909501589608,
   -0.143772296441519,
   0.635558093258208,
  

In [3]:
sc = SparkContext(appName='creditcard_lowlevel', conf=config)

your 131072x1 screen size is bogus. expect trouble
25/04/11 23:50:22 WARN Utils: Your hostname, DESKTOP-0H87CFM resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/11 23:50:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/11 23:50:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/11 23:50:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
#Split data into training and testing sets

train_size = 0.8
train_rdd, test_rdd = data_rdd.randomSplit([train_size, 1 - train_size], seed=24)

# Compute mean and std for Time and Amount from training data
#times = train_rdd.map(lambda x: x[0][0])  # Time is the first feature
#amounts = train_rdd.map(lambda x: x[0][-1])  # Amount is the last feature
#mean_time, std_time = times.mean(), times.stdev()
#mean_amount, std_amount = amounts.mean(), amounts.stdev()

# Function to scale Time and Amount
#def scale_features(features, mean_time, std_time, mean_amount, std_amount):
#    scaled_time = (features[0] - mean_time) / std_time
#    scaled_amount = (features[-1] - mean_amount) / std_amount
#    return [scaled_time] + features[1:-1] + [scaled_amount]

# Apply scaling to training and testing data
#train_rdd_scaled = train_rdd.map(lambda x: (scale_features(x[0], mean_time, std_time, mean_amount, std_amount), x[1]))
#test_rdd_scaled = test_rdd.map(lambda x: (scale_features(x[0], mean_time, std_time, mean_amount, std_amount), x[1]))

## Define function for Logistic Regression

In [6]:
import math

class LogisticRegression:
    def __init__(self, learning_rate=0.01, num_iterations=100):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.weights = None

    def _add_bias(self, features):
        return features + [1.0]

    def dot_product(self, features, weights):
        return sum(f * w for f, w in zip(features, weights))

    def sigmoid(self, z):
        try:
            return 1.0 / (1.0 + math.exp(-z))
        except OverflowError:
            return 0.0 if z < 0 else 1.0

    def compute_gradient(self, features, label, weights):
        features = self._add_bias(features)
        prediction = self.sigmoid(self.dot_product(features, weights))
        error = prediction - label
        return [error * x for x in features]

    def predict(self, features):
        features = self._add_bias(features)
        prob = self.sigmoid(self.dot_product(features, self.weights))
        return 1.0 if prob >= 0.5 else 0.0

    def fit(self, data_rdd):
        first_record = self._add_bias(data_rdd.first()[0])
        n_features = len(first_record)
        self.weights = [0.0] * n_features

        for _ in range(self.num_iterations):
            gradient = data_rdd.map(
                lambda x: self.compute_gradient(x[0], x[1], self.weights)
            ).reduce(
                lambda a, b: [a_i + b_i for a_i, b_i in zip(a, b)]
            )
            self.weights = [w - self.learning_rate * g for w, g in zip(self.weights, gradient)]

    @property
    def coefficients(self):
        if self.weights:
            return self.weights[:-1]
        return None

    @property
    def intercept(self):
        if self.weights:
            return self.weights[-1]
        return None


## Fit the model

In [7]:
learning_rate = 0.01
num_iterations = 100

log_reg = LogisticRegression(learning_rate=learning_rate, num_iterations=num_iterations)
log_reg.fit(test_rdd)

In [8]:
# Display model coefficients and intercept
print("Coefficients: ", log_reg.coefficients)
print("Intercept: ", log_reg.intercept)

Coefficients:  [-18558517.814999912, -451.9070833575454, 323.1674373614058, -672.8055773984966, 420.7320930536979, -283.4986506079108, -161.39010681198297, -560.3240408390903, 97.11641769025482, -234.74305450674441, -553.9357980152168, 358.9380935161544, -603.3281433715642, -14.167436175648996, -665.9935910140988, -17.68501231608554, -406.01961603100364, -657.1762738153172, -216.51626850515333, 71.08733986780864, 21.791544144273484, 76.03649752811747, 4.797914556075549, -13.5525158444723, -7.889537254028329, -1.6964800574122842, 2.2398545271871435, 3.9624407886994786, 4.987548066183656, -18603.410049999766]
Intercept:  -192.24999999999932


In [9]:
def compute_accuracy(predictions_rdd):
    correct = predictions_rdd.filter(lambda x: x[0] == x[1]).count()
    total = predictions_rdd.count()
    return correct / total if total > 0 else 0.0

def compute_precision(predictions_rdd, label):
    true_positive = predictions_rdd.filter(lambda x: x[0] == label and x[1] == label).count()
    predicted_positive = predictions_rdd.filter(lambda x: x[0] == label).count()
    return true_positive / predicted_positive if predicted_positive > 0 else 0.0

def compute_recall(predictions_rdd, label):
    true_positive = predictions_rdd.filter(lambda x: x[0] == label and x[1] == label).count()
    actual_positive = predictions_rdd.filter(lambda x: x[1] == label).count()
    return true_positive / actual_positive if actual_positive > 0 else 0.0

In [11]:
#Evaluate the model on training data
predictions_train = train_rdd.map(lambda p: (log_reg.predict(p[0]), p[1]))

print("Train Accuracy:", compute_accuracy(predictions_train))
print("Train Precision: ", [compute_precision(predictions_train, label) for label in [0, 1]])
print("Train Recall: ", [compute_recall(predictions_train, label) for label in [0, 1]])

Train Accuracy: 0.9982481328784626


Train Precision:  [0.9982481328784626, 0.0]


Train Recall:  [1.0, 0.0]


In [12]:
#Evaluate the model on test data
predictions_test = test_rdd.map(lambda p: (log_reg.predict(p[0]), p[1]))

print("Test Accuracy:", compute_accuracy(predictions_test))
print("Test Precision: ", [compute_precision(predictions_test, label) for label in [0, 1]])
print("Test Recall: ", [compute_recall(predictions_test, label) for label in [0, 1]])

Test Accuracy: 0.9983698510078878


Test Precision:  [0.9983698510078878, 0.0]


Test Recall:  [1.0, 0.0]


In [ ]:
#Stop the Spark session
spark.stop()